In [42]:
import requests
import json
import csv
import datetime as dt
import pandas as pd
import numpy as np
import re 
from datetime import datetime
import time
import os
from collections import OrderedDict
from gensim.summarization.summarizer import summarize
import itertools 

In [43]:
api_key="juEydPi9AVH26V9AGTfv"

#Getting the response of the api call into a json file
response=requests.post("https://api.figure-eight.com/v1/jobs.json?key="+api_key)

In [44]:
#response status code
print(response.status_code)

200


In [45]:
#response content, job details
print(response.content)

b'{"id":1394781,"options":{"logical_aggregation":true,"mail_to":"w.huang-10@student.tudelft.nl","req_ttl_in_seconds":1800},"title":null,"secret":"yR7fFhY5EhEQbYNlFeAlxbP2iXwB/j2A1/BjrrllJNG0","project_number":null,"alias":null,"judgments_per_unit":3,"units_per_assignment":5,"pages_per_assignment":1,"max_judgments_per_worker":null,"gold_per_assignment":1,"minimum_account_age_seconds":null,"execution_mode":"worker_ui_remix","payment_cents":10,"design_verified":true,"public_data":false,"variable_judgments_mode":"none","max_judgments_per_unit":null,"expected_judgments_per_unit":null,"min_unit_confidence":null,"units_remain_finalized":null,"auto_order_timeout":null,"auto_order_threshold":4,"completed_at":null,"state":"unordered","auto_order":false,"webhook_uri":null,"send_judgments_webhook":null,"language":"en","minimum_requirements":null,"desired_requirements":null,"max_work_per_network":null,"copied_from":null,"created_at":"2019-06-16T23:19:45+00:00","updated_at":"2019-06-16T23:19:45+00:0

In [46]:
#Creating task with title, instructions and no data
job_title = "Summary grammar task - Myth"
instructions="<h1>Overview</h1><p>Please choose the summary you previously selected and perform a grammar check. </p><hr><h1>Steps</h1><p>1. Choose the summary you selected in the last task.</p><p>2. Read the summary and identify grammatical/syntax errors.</p><p>3. Make necessary changes of the summary in the textbox. If there are no detected grammatical errors, click the 'I didn't spot an errIor' checkbox.</p>"
instructions+="<p>5. Submit your work.</p><hr><h1>Previous summary</h1><p>The previous summary is:</p><p>Finally, Zeus bestowed  two gifts on Pandora. But the contents, Zeus told her,  were not for mortal eyes. Over time, Pandora became more and more  obsessed with the box. As she wept, Pandora became aware of a sound echoing  from within the box.</p>"

In [47]:
#setup the request url
request_url = "https://api.figure-eight.com/v1/jobs.json"
headers = {'content-type': 'application/json'}

In [48]:
cml="""
<cml:radios validates="required" label="Do you see any errors in this summary?">
  <cml:radio label="I spot errors and will corrected it below." />
  <cml:radio label="I didn't spot an error." />
</cml:radios>
<cml:text label="In case there are errors in the selected summary, please correct it in the textbox:" name="improved_grammar." aggregation="all" gold="true" />"""

In [49]:
#Setup the payload object
payload = {
'key': api_key,
'job':{
'title': job_title,
'instructions': instructions,
'cml': cml
}
}

response=requests.post(request_url, data=json.dumps(payload), headers=headers)
#print(response.content)

In [50]:
#find job id and save it
workJson = json.loads(response.content)
job_id = workJson['id']
print(job_id)

1394782


In [51]:
#Updating the task we had created before by adding data row by row (unit by unit)
API_KEY = "juEydPi9AVH26V9AGTfv"
sum_workers = "Finally, Zeus bestowed  two gifts on Pandora. But the contents, Zeus told her,  were not for mortal eyes. Over time, Pandora became more and more  obsessed with the box. As she wept, Pandora became aware of a sound echoing  from within the box."

data = {"choice": sum_workers}

request_url = "https://api.figure-eight.com/v1/jobs/{}/units.json".format(job_id)
headers = {'content-type': 'application/json'}

payload = {
'key': API_KEY,
'data': data
}
response=requests.post(request_url, data=json.dumps(payload), headers=headers)